# 👊 CHAPTER 15 Autoencoder(오토인코더)

#### 🙋 Chapter Manager : 장승준  🙎 Contents Add :   남창헌             

## 👊 Autoencoder    
              
### &nbsp;&nbsp;𝐎 정의    

   - label 되어 있지 않은 Train Data를 사용하여 (Unsupervised Learning) 입력 data의 효율적인 표현인    
     coding을 학습할 수 있는 ANN(Artificial Neural Network)
          
     ※ coding : 컴퓨터 프로그래밍이 아닌 일반적인 부호화를 의미 (주어진 정보를 어떤 표준적인 형태로 변환하거나 거꾸로 변환함.)

### &nbsp;&nbsp;𝐎 특징    

   - 일반적으로 입력보다 훨씬 낮은 차원을 가지므로 autoencoder가 차원 축소에 유용     
   - 강력한 특성 추출기처럼 작동하기 때문에 심층 신경망(DNN)의 unspervised learning prior train 사용 가능    
   - Train data와 매우 비슷한 새로운 data를 생성 가능 => Generative Model이라 칭함    
     ex. 얼굴 사진을 사용해 autoencoder를 훈련시키면 이 모델은 새로운 얼굴을 생성 가능

### &nbsp;&nbsp;𝐎 원리
   - Autoencoder가 학습하는 것은 단순히 입력을 출력으로 복사하는 것
   - Network에 여러 방법으로 제약을 가해 오히려 어려운 작업으로 만듦  
     ex. 내부표현의 크기를 제한하거나 입력에 잡음을 추가한 후 원본 입력을 복원할 수 있도록 network를 훈련시킴    
            
     ※ 내부 표현의크기를 제한 : hidden layer에 있는 neural 수를 input layer보다 적게한다는 뜻
        
1. 입력을 받아 효율적인 내부 표현으로 바꾸고 입력과 매우 가까워 보이는 어떤 것을 출력 output layer의 neural 수가   
   input layer와 동일하다는 것을 제외하면 일반적으로 autoencoder는 MLP(Multi Layer Perceptron)과 동일한 구조                 
         
            
2. autoencoder가 입력을 재구성하기 때문에 출력을 reconstruction(재구성)이라고 부름  
   cost function은 재구성이 입력과 다를 때 model에 벌점을 부과하는 reconstruction loss(재구성 손실)을 포함    
       
         
3. 내부 표현이 입력 data보다 저차원이기 때문에 이런 autoencoder를 Undercomplete(과소완전)라 함    
   Undercomplete autoencoder는 입력을 coding으로 간단히 복사할 수 없으며, 입력과 똑같은 것을 출력하기 위한 다른 방법을 찾아야하며,   
   이는 입력 data에서 가장 중요한 특성을 학습하도록 만듦(그리고 중요하지 않은 것은 버림)  

### &nbsp;&nbsp;𝐎 구성
- encoder와 decoder로 구성
- encoder : 입력을 내부로 표현으로 바꾸는 역할 (recognition network, 인지 네트워크)
- decoder : 내부 표현을 출력으로 바꾸는 역할 ( generative network, 생성 네트워크)

## 👊 Undercomplete(과소완전) 선형 autoencoder로 PCA 수행

- autoencoder가 선형 활성화 함수만 사용하고 cost function이 MSE(평균 제곱 오차)라면 이는 결국 PCA를 수행하는 것으로 볼 수 있음    
  PCA는 분산이 가장 큰 방향(주성분, principal component)를 찾음 (원본 데이터와 투영 거리(euclidean distance)를 가장 작게 만드는 성분을 찾는 것)    
        
  따라서 투영 거리를 가장 작게 만드는 것은 MSE를 최소화하려는 autoencoder의 목적과 동일    
  But PCA로 찾은 주성분과 autoencoder의 coding은 같지 않음

In [1]:
import numpy as np
import pandas as pd
import numpy.random as rnd
import sys
import tensorflow as tf
from functools import partial

n_inputs = 3 # 3D 입력
n_hidden = 2 # 2D 입력
n_outputs = n_inputs

learning_rate = 0.01

X = tf.placeholder(tf.float32, shape = [None, n_inputs])
hidden = tf.layers.dense(X, n_hidden)
outputs = tf.layers.dense(hidden, n_outputs)
reconstruction_loss = tf.reduce_mean(tf.square(outputs - X)) # MSE

optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(reconstruction_loss)

init = tf.global_variables_initializer()

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


- 출력의 개수가 입력의 개수와 동일
- 단순한 PCA를 수행하기 위해서는 활성화 함수(activation function)를 사용하지 않으며( 즉, 모든 neural이 선형) ,cost function은 MSE.  

In [2]:
X_train, X_test = [...] # 데이터 셋을 로드

n_iterations = 1000
codings = hidden  # coding을 만드는 hidden layer를 출력  

with tf.Session() as sess :
    init.run()
    for iteration in range(n_iterations) :
        training_op.run( feed_dict = { X : X_train}) # label이 없음( unspervised learning )
    codings_val = codings.eval(feed_dict = {X : X_test})

ValueError: not enough values to unpack (expected 2, got 1)

=> 원본 data set과 autoencoder의 hidden layer의 출력( 즉, coding layer)  
autoencoder는 data에 있는 분산이 가능한 많이 보존되도록 data를 투영할 최상의 2D 평면을 찾음

## 👊 Stacked(적층) Autoencoder (Deep Autoencoder, 심층 오토인코더)

### &nbsp;&nbsp;𝐎 구조

In [ ]:
from IPython.display import Image

Image(filename = "Ch 15. Stacked Autoencoder 구조.JPG", width = 600, height = 600)

### &nbsp;&nbsp;𝐎 특징
- Autoencoder도 여러 개의 hidden layer를 가질 수 있으면서 나온 Stacked Autoencoder   
  layer를 더 추가하면 autoencoder가 더 복잡한 coding을 학습 가능 (But autoencoder가 너무 강력하게 되지 않도록 주의)  
    
        
- encoder가 너무 강력해서 각각의 입력  data를 임의의 한 숫자로 mapping하도록 학습, decoder는 역으로 mapping 하는 것을 학습했다고 하면  
  훈련 data는 완벽하게 재구성을 하겠지만 유용한 데이터 표현을 학습하지는 못할 것   
       
          
- Stacked Autoencoder의 구조는 전형적으로 hidden layer(coding layer)을 기준으로 대칭! (샌드위치 모양)  

#### - Stacked Autoencoder는 평범한 심층 MLP과 매우 비슷하게 구현 가능    
   - He 초기화, ELU 활성화 함수, $l$2 규제를 사용해 MNIST를 위한 Stacked Encoder (label이 없는 것을 제외하면 앞 chapter에서 다룬 코드들과 유사)  

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

In [ ]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()

n_inputs = 28 * 28 #MNIST
n_hidden1 = 300
n_hidden2 = 150  # codings
n_hidden3 = n_hidden1
n_outputs = n_inputs

learning_rate = 0.01
l2_reg = 0.0001

X = tf.placeholder(tf.float32, shape=[None, n_inputs])

he_init = tf.contrib.layers.variance_scaling_initializer() # He initialization
#Equivalent to:
#he_init = lambda shape, dtype=tf.float32: tf.truncated_normal(shape, 0., stddev=np.sqrt(2/shape[0]))
l2_regularizer = tf.contrib.layers.l2_regularizer(l2_reg)
my_dense_layer = partial(tf.layers.dense,
                         activation=tf.nn.elu,
                         kernel_initializer=he_init,
                         kernel_regularizer=l2_regularizer)

hidden1 = my_dense_layer(X, n_hidden1)
hidden2 = my_dense_layer(hidden1, n_hidden2)
hidden3 = my_dense_layer(hidden2, n_hidden3)
outputs = my_dense_layer(hidden3, n_outputs, activation=None)

reconstruction_loss = tf.reduce_mean(tf.square(outputs - X))

reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
loss = tf.add_n([reconstruction_loss] + reg_losses)

optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()
saver = tf.train.Saver() # not shown in the book

In [ ]:
n_epochs = 5
batch_size = 150

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):  # batch_size
        n_batches = mnist.train.num_examples // batch_size   
        for iteration in range(n_batches):
            print("\r{}%".format(100 * iteration // n_batches), end="") # not shown in the book
            sys.stdout.flush()                                          # not shown
            # shuffle_batch() 함수정의는 github를 참고
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch})
        loss_train = reconstruction_loss.eval(feed_dict={X: X_batch})   # not shown
        print("\r{}".format(epoch), "Train MSE:", loss_train)           # not shown
        saver.save(sess, "./my_model_all_layers.ckpt")                  # not shown

In [ ]:
def show_reconstructed_digits(X, outputs, model_path = None, n_test_digits = 2):
    with tf.Session() as sess:
        if model_path:
            saver.restore(sess, model_path)
        X_test = mnist.test.images[:n_test_digits]
        outputs_val = outputs.eval(feed_dict={X: X_test})

    fig = plt.figure(figsize=(8, 3 * n_test_digits))
    for digit_index in range(n_test_digits):
        plt.subplot(n_test_digits, 2, digit_index * 2 + 1)
        plot_image(X_test[digit_index])
        plt.subplot(n_test_digits, 2, digit_index * 2 + 2)
        plot_image(outputs_val[digit_index])

In [ ]:
import os
import matplotlib.pyplot as plt
%matplotlib inline

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "autoencoders"

def save_fig(fig_id, tight_layout=True):
    #path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(fig_id + ".png", format='png', dpi=300)

def plot_image(image, shape=[28, 28]):
    plt.imshow(image.reshape(shape), cmap="Greys", interpolation="nearest")
    plt.axis("off")
    
show_reconstructed_digits(X, outputs, "./my_model_all_layers.ckpt")
save_fig("reconstruction_plot")

### &nbsp;&nbsp;𝐎 Weight(가중치) 묶기
- Stacked Autoencoder가 완벽하게 대칭일 때(샌드위치 형태) 일반적으로 decoder의 가중치와 encoder의 가중치를 묶음  
- Decoder와 Encoder의 가중치를 묶으면 Model에 있는 가중치의 갯수를 절반으로 줄여서 훈련 속도를 높이고 overfitting의 위험을 줄여줌   
    
       
- 임의의 Autoencoder가 (input layer은 제외한) N개의 층을 가지고 있고 $W_{L}$이 L번째 층의 가중치를 나타낸다고 했을 때 예를 들어 1은 첫 번째 hidden layer, $ \frac{N}{2}$은 coding layer, $N$은 output layer)     
    
       
- Decoder layer의 가중치 : $W_{N-L+1} = W_{L}^{T}$ ($L = 1,2, ... , \frac{N}{2}$)  
  (weights_decoder = tf.transpose(weights_encoder))
- tensorflow에서 dense() 함수를 사용해 가중치를 묶으려면 조금 복잡하기에 직접 layer를 정의하는 것이 더 쉬움.

In [ ]:
reset_graph()

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 150  # codings
n_hidden3 = n_hidden1
n_outputs = n_inputs

learning_rate = 0.01
l2_reg = 0.0005

In [ ]:
activation = tf.nn.elu
regularizer = tf.contrib.layers.l2_regularizer(l2_reg)
initializer = tf.contrib.layers.variance_scaling_initializer()

X = tf.placeholder(tf.float32, shape=[None, n_inputs])

weights1_init = initializer([n_inputs, n_hidden1])
weights2_init = initializer([n_hidden1, n_hidden2])

weights1 = tf.Variable(weights1_init, dtype=tf.float32, name="weights1")
weights2 = tf.Variable(weights2_init, dtype=tf.float32, name="weights2")
weights3 = tf.transpose(weights2, name="weights3")  # tied weights
weights4 = tf.transpose(weights1, name="weights4")  # tied weights

biases1 = tf.Variable(tf.zeros(n_hidden1), name="biases1")
biases2 = tf.Variable(tf.zeros(n_hidden2), name="biases2")
biases3 = tf.Variable(tf.zeros(n_hidden3), name="biases3")
biases4 = tf.Variable(tf.zeros(n_outputs), name="biases4")

hidden1 = activation(tf.matmul(X, weights1) + biases1)
hidden2 = activation(tf.matmul(hidden1, weights2) + biases2)
hidden3 = activation(tf.matmul(hidden2, weights3) + biases3)
outputs = tf.matmul(hidden3, weights4) + biases4

reconstruction_loss = tf.reduce_mean(tf.square(outputs - X))
reg_loss = regularizer(weights1) + regularizer(weights2)
loss = reconstruction_loss + reg_loss

optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()

- weights3와 weights4는 변수로 선언되지 않았고 각각 weights2와 weights1의 전치입니다.(서로 묶여있음)
- 변수가 아니기 때문에 규제에 사용되지 않음. weights1과 weights만 규제
- 편향은 묶지도 않고 규제하지도 않음

In [ ]:
saver = tf.train.Saver()

In [ ]:
n_epochs = 5
batch_size = 150

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        n_batches = mnist.train.num_examples // batch_size
        for iteration in range(n_batches):
            print("\r{}%".format(100 * iteration // n_batches), end="")
            sys.stdout.flush()
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch})
        loss_train = reconstruction_loss.eval(feed_dict={X: X_batch})
        print("\r{}".format(epoch), "Train MSE:", loss_train)
        saver.save(sess, "./my_model_tying_weights.ckpt")

In [ ]:
show_reconstructed_digits(X, outputs, "./my_model_tying_weights.ckpt")

### &nbsp;&nbsp;𝐎 한 번에 한 층씩 훈련
전체 autoencoder를 훈련 시키기보단 한 번에 autoencoder 하나를 훈련하고 이를 쌓아올려 한 개의 Stacked Autoencoder 하나를 훈련하고 이를 쌓아올려 한 개의 Stacked Autoencoder를 만드는 것이 더 빠름

In [ ]:
from IPython.display import Image
Image(filename = "Ch 15. 한 번에 autoencoder 한 개씩 훈련하기.jpg", width = 600, height = 600)

- 이러한 다단계 훈련 Algorithm을 구현하는 가장 간단한 방법은   
              
 1. 하나의 autoencoder 훈련이 끝나면 훈련 set을 network에 통과시켜 hidden layer의 출력을 저장
 2. 1.에서의 출력은 다음 autoencoder의 훈련 set이 됨
 3. 모든 autoencoder가 이런 식으로 훈련을 마치며(1과 2를 반복) 각 autoencoder의 가중치와 편향을 복사해서 stacked autoencoder를 만듦

In [ ]:
reset_graph()

from functools import partial

def train_autoencoder(X_train, n_neurons, n_epochs, batch_size,
                      learning_rate = 0.01, l2_reg = 0.0005, seed=42,
                      hidden_activation=tf.nn.elu,
                      output_activation=tf.nn.elu):
    graph = tf.Graph()
    with graph.as_default():
        tf.set_random_seed(seed)

        n_inputs = X_train.shape[1]

        X = tf.placeholder(tf.float32, shape=[None, n_inputs])
        
        my_dense_layer = partial(
            tf.layers.dense,
            kernel_initializer=tf.contrib.layers.variance_scaling_initializer(),
            kernel_regularizer=tf.contrib.layers.l2_regularizer(l2_reg))

        hidden = my_dense_layer(X, n_neurons, activation=hidden_activation, name="hidden")
        outputs = my_dense_layer(hidden, n_inputs, activation=output_activation, name="outputs")

        reconstruction_loss = tf.reduce_mean(tf.square(outputs - X))

        reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
        loss = tf.add_n([reconstruction_loss] + reg_losses)

        optimizer = tf.train.AdamOptimizer(learning_rate)
        training_op = optimizer.minimize(loss)

        init = tf.global_variables_initializer()

    with tf.Session(graph=graph) as sess:
        init.run()
        for epoch in range(n_epochs):
            n_batches = len(X_train) // batch_size
            for iteration in range(n_batches):
                print("\r{}%".format(100 * iteration // n_batches), end="")
                sys.stdout.flush()
                indices = rnd.permutation(len(X_train))[:batch_size]
                X_batch = X_train[indices]
                sess.run(training_op, feed_dict={X: X_batch})
            loss_train = reconstruction_loss.eval(feed_dict={X: X_batch})
            print("\r{}".format(epoch), "Train MSE:", loss_train)
        params = dict([(var.name, var.eval()) for var in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)])
        hidden_val = hidden.eval(feed_dict={X: X_train})
        return hidden_val, params["hidden/kernel:0"], params["hidden/bias:0"], params["outputs/kernel:0"], params["outputs/bias:0"]

Now let's train two Autoencoders. The first one is trained on the training data, and the second is trained on the previous Autoencoder's hidden layer output:

In [ ]:
hidden_output, W1, b1, W4, b4 = train_autoencoder(mnist.train.images, n_neurons=300, n_epochs=4, batch_size=150,
                                                  output_activation=None)
_, W2, b2, W3, b3 = train_autoencoder(hidden_output, n_neurons=150, n_epochs=4, batch_size=150)

Finally, we can create a Stacked Autoencoder by simply reusing the weights and biases from the Autoencoders we just trained:

In [ ]:
reset_graph()

n_inputs = 28*28

X = tf.placeholder(tf.float32, shape=[None, n_inputs])
hidden1 = tf.nn.elu(tf.matmul(X, W1) + b1)
hidden2 = tf.nn.elu(tf.matmul(hidden1, W2) + b2)
hidden3 = tf.nn.elu(tf.matmul(hidden2, W3) + b3)
outputs = tf.matmul(hidden3, W4) + b4

In [ ]:
show_reconstructed_digits(X, outputs)

### &nbsp;&nbsp;𝐎 Stacked Autoencoder를 훈련하는 하나의 Graph

In [ ]:
from IPython.display import Image
Image(filename = "Ch 15. Stacked Autoencoder를 훈련하는 하나의 graph.JPG", width = 600, height = 600)

1. 그래프 중앙 부분은 전체 Stacked Autoencoder (훈련이 끝난 뒤에 사용)
2. 그래프 왼쪽부분은 훈련 단계 1을 실행하기 위해 필요한 연산. 
3. 여기에선 hidden layer 2와 3을 그냥 지나치는 output layer을 만듦. -> 가중치와 편향을 stacked autoencoder의 output layer와 공유
4. 3.위에 출력을 가능한 한 입력과 가깝게 만들기 위한 훈련 연산이 있으므로 'hidden layer 1'과 'output layer 2'의 가중치와 편향을 훈련시킴   
   ( 즉, 첫 번째의 autoencoder )   
5. 그래프의 오른쪽 부분은 훈련 단계 2를 실행하기 위해 필요한 연산. 
6. hidden layer 3의 출력을 hidden layer 1의 출력과 가능한 한 가깝게 만들기 위한 훈련 연산이 추가됨. 
7. 단계 2를 실행하는 동안에는 hidden layer 1을 동결해야 함 ->  hidden layer 2와 hidden layer 3의 가중치와 편향을 훈련 시킴    
   ( 즉, 두 번째의 autoencoder )

In [ ]:
reset_graph()

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 150  # codings
n_hidden3 = n_hidden1
n_outputs = n_inputs

learning_rate = 0.01
l2_reg = 0.0001

activation = tf.nn.elu
regularizer = tf.contrib.layers.l2_regularizer(l2_reg)
initializer = tf.contrib.layers.variance_scaling_initializer()

X = tf.placeholder(tf.float32, shape=[None, n_inputs])

weights1_init = initializer([n_inputs, n_hidden1])
weights2_init = initializer([n_hidden1, n_hidden2])
weights3_init = initializer([n_hidden2, n_hidden3])
weights4_init = initializer([n_hidden3, n_outputs])

weights1 = tf.Variable(weights1_init, dtype=tf.float32, name="weights1")
weights2 = tf.Variable(weights2_init, dtype=tf.float32, name="weights2")
weights3 = tf.Variable(weights3_init, dtype=tf.float32, name="weights3")
weights4 = tf.Variable(weights4_init, dtype=tf.float32, name="weights4")

biases1 = tf.Variable(tf.zeros(n_hidden1), name="biases1")
biases2 = tf.Variable(tf.zeros(n_hidden2), name="biases2")
biases3 = tf.Variable(tf.zeros(n_hidden3), name="biases3")
biases4 = tf.Variable(tf.zeros(n_outputs), name="biases4")

hidden1 = activation(tf.matmul(X, weights1) + biases1)
hidden2 = activation(tf.matmul(hidden1, weights2) + biases2)
hidden3 = activation(tf.matmul(hidden2, weights3) + biases3)
outputs = tf.matmul(hidden3, weights4) + biases4

reconstruction_loss = tf.reduce_mean(tf.square(outputs - X))

In [ ]:
optimizer = tf.train.AdamOptimizer(learning_rate)

with tf.name_scope("phase1"):
    phase1_outputs = tf.matmul(hidden1, weights4) + biases4  # bypass hidden2 and hidden3
    phase1_reconstruction_loss = tf.reduce_mean(tf.square(phase1_outputs - X))
    phase1_reg_loss = regularizer(weights1) + regularizer(weights4)
    phase1_loss = phase1_reconstruction_loss + phase1_reg_loss
    phase1_training_op = optimizer.minimize(phase1_loss)

with tf.name_scope("phase2"):
    phase2_reconstruction_loss = tf.reduce_mean(tf.square(hidden3 - hidden1))
    phase2_reg_loss = regularizer(weights2) + regularizer(weights3)
    phase2_loss = phase2_reconstruction_loss + phase2_reg_loss
    train_vars = [weights2, biases2, weights3, biases3]
    phase2_training_op = optimizer.minimize(phase2_loss, var_list=train_vars) # freeze hidden1

In [ ]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [ ]:
training_ops = [phase1_training_op, phase2_training_op]
reconstruction_losses = [phase1_reconstruction_loss, phase2_reconstruction_loss]
n_epochs = [4, 4]
batch_sizes = [150, 150]

with tf.Session() as sess:
    init.run()
    for phase in range(2):
        print("Training phase #{}".format(phase + 1))
        for epoch in range(n_epochs[phase]):
            n_batches = mnist.train.num_examples // batch_sizes[phase]
            for iteration in range(n_batches):
                print("\r{}%".format(100 * iteration // n_batches), end="")
                sys.stdout.flush()
                X_batch, y_batch = mnist.train.next_batch(batch_sizes[phase])
                sess.run(training_ops[phase], feed_dict={X: X_batch})
            loss_train = reconstruction_losses[phase].eval(feed_dict={X: X_batch})
            print("\r{}".format(epoch), "Train MSE:", loss_train)
            saver.save(sess, "./my_model_one_at_a_time.ckpt")
    loss_test = reconstruction_loss.eval(feed_dict={X: mnist.test.images})
    print("Test MSE:", loss_test)

1. 단계1은 hidden layer 2와 3을 건너뛰는 output layer를 만들고, 이 출력과 입력 사이의 거리를 (+ 일부 규제) 최소화하는 훈련 연산을 만듦
    
      
2. 단계2는 hidden layer3과 hidden layer1 사이의 거리를(+ 일부 규제) 최소화하는데 필요한 연산 추가   
가장 중요한 점은 minimize() method에 훈련될 변수 목록을 넣는데 weights1과 biases1은 제외 (=> hidden layer 1을 동결시키는 효율적인 방법)

- Total
  1. 단계 1의 훈련 연산에 대해 여러 번 epoch를 실행
  2. 단계 2의 훈련 연산에 대해 조금 더 많은 epoch를 실행

- Tip
  1. hidden layer1이 단계 2에서 동결되기 때문에 각 훈련 sample에 대한 hidden layer의 출력은 동일.
  2. hidden layer 1의 출력을 매 epoch마다 다시 계산하지 않도록 하려면 단계 1의 끝에서 전체 훈련 set에 대해 hidden layer 1의 출력을 계산
  3. 단계 2동안 hidden layer 2의 캐싱된 출력을 입력으로 주입  
### => 훈련 속도를 상당히 높일 수 있음

### &nbsp;&nbsp;𝐎 Cache the frozen layer outputs

In [ ]:
training_ops = [phase1_training_op, phase2_training_op]
reconstruction_losses = [phase1_reconstruction_loss, phase2_reconstruction_loss]
n_epochs = [4, 4]
batch_sizes = [150, 150]

with tf.Session() as sess:
    init.run()
    for phase in range(2):
        print("Training phase #{}".format(phase + 1))
        if phase == 1:
            hidden1_cache = hidden1.eval(feed_dict={X: mnist.train.images})
        for epoch in range(n_epochs[phase]):
            n_batches = mnist.train.num_examples // batch_sizes[phase]
            for iteration in range(n_batches):
                print("\r{}%".format(100 * iteration // n_batches), end="")
                sys.stdout.flush()
                if phase == 1:
                    indices = rnd.permutation(mnist.train.num_examples)
                    hidden1_batch = hidden1_cache[indices[:batch_sizes[phase]]]
                    feed_dict = {hidden1: hidden1_batch}
                    sess.run(training_ops[phase], feed_dict=feed_dict)
                else:
                    X_batch, y_batch = mnist.train.next_batch(batch_sizes[phase])
                    feed_dict = {X: X_batch}
                    sess.run(training_ops[phase], feed_dict=feed_dict)
            loss_train = reconstruction_losses[phase].eval(feed_dict=feed_dict)
            print("\r{}".format(epoch), "Train MSE:", loss_train)
            saver.save(sess, "./my_model_cache_frozen.ckpt")
    loss_test = reconstruction_loss.eval(feed_dict={X: mnist.test.images})
    print("Test MSE:", loss_test)

### &nbsp;&nbsp;𝐎 Visualizing the Reconstructions
- Autoencoder가 제대로 훈련되었는지 확인하는 법
  1. 입력과 출력을 비교
    - 입력과 출력은 매우 비슷해야 하고, 차이가 있다면 중요한 부분이 아니여야 함.

In [ ]:
# 임의로 선택한 두 개의 숫자로 이용해 재구성을 확인

n_test_digits = 2
X_test = mnist.test.images[:n_test_digits]

with tf.Session() as sess:
    saver.restore(sess, "./my_model_one_at_a_time.ckpt") # not shown in the book
    outputs_val = outputs.eval(feed_dict = { X : X_test[:n_test_digits]})
    
def plot_image( image, shape = [28, 28]) :
    plt.imshow(image.reshape(shape) , cmap = "Greys", interpolation = "nearest")
    plt.axis("off")
    
for digit_index in range(n_test_digits) :
    plt.subplot(n_test_digits, 2, digit_index * 2 + 1)
    plot_image(X_test[digit_index])
    plt.subplot(n_test_digits, 2, digit_index * 2 + 2)
    plot_image(outputs_val[digit_index])

### &nbsp;&nbsp;𝐎 Feature Visualization (Visualizing the extracted features)
Autoencoder가 특정 특성을 학습했으므로 이 특정 특성을 확인해보고 싶을 것
1. 가장 간단한 기법으로 hidden layer에 있는 각 뉴런을 가장 크게 활성화시키는 훈련 sample을 찾는 것.   
=> 최상단의 hidden layer에 유용, 비교적 큰 특성들을 잡아내기에 이를 포함한 sample을 찾기 쉬움    
ex. 한 뉴런이 고양이가 있는 image를 보았을 때 강하게 활성화가 되었다면 이 뉴런을 활성화 시키는 image에는 대부분 고양이가 들어있을 것.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;But 하위층은 특성의 크기가 작고 추상적이기에 뉴런이 무엇때문에 활성화가 되었는지 이해하기 어려움 따라서, 이러한 기법이 적합하지 않음

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_one_at_a_time.ckpt") # not shown in the book
    weights1_val = weights1.eval()

for i in range(5):
    plt.subplot(1, 5, i + 1)
    plot_image(weights1_val.T[i])

save_fig("extracted_features_plot") # not shown
plt.show()                          # not shown

2. Random한 입력 image를 autoencoder에 주입하고, 관심 있는 neurons의 활성화를 관찰해서 이 neurons이 더 활성화되는 방향으로 image를 수정하도록 backpropagation를 수행   
-> 이를 여러 번 반복하면(gradient ascent(경사상승법)를 수행하여) image는 점차(그 neurons을 위해) 놀라운 방향으로 바뀌게 됨.   
이는 neurons이 바라보는 입력의 종류를 시각화하는데 유용.

## 👊 Unsupervised pretraining (Stacked Autoencoder를 사용한 Unsupervised pretraining)

### &nbsp;&nbsp;𝐎 적용    
     
1. label 되어 있는 훈련 data가 많지 않은 복잡한 supervised learning 문제를 다뤄야한다면,  
   - 비슷한 문제에 학습된 신경망을 찾아 하위층을 재사용하는 것.  
   => 저수준의 특성을 학습할 필요가 없기에 적은 훈련 data만 사용해도 고성능 model 훈련이 가능
   즉, 기존의 network에서 학습한 특성 감지 기능을 재사용하는 것
  
2. 대부분 label 되어 있지 않은 대량의 data set이 있다면,  
   1. 먼저 전체 data를 사용해 Stacked Autoencoder를 훈련 시킬 수 있음
   2. Autoencoder의 하위층을 재사용해 실제 문제를 해결하기 위한 신경망을 위해 Unsupervised Learning Prior Train을 수행하는 Stack Autoencoder를 사용  
   3. 535p 그림 15-8은 분류에 대한 neural network을 위해 unsupervised learning prior train을 수행하는 stacked autoencoder를 사용하는 법을 보여줌

#### Autoencoder를 사용한 Unspervised Pretrain

In [ ]:
from IPython.display import Image
Image(filename = "Ch 15. Autoencoder를 사용한 Unspervised pretrain.jpg", width  = 600, height = 600)

### &nbsp;&nbsp;𝐎 특징
- Stacked Autoencoder는 한 번에 한  layer씩 훈련되므로 분류기를 훈련할 때,   
  label된 훈련 data가 많지 않으면서 사전훈련된 layer을 동결하는 것이 좋음  
  (적어도 가장 하위층 하나는)

- 대량의 label 되지 않은 데이터는 만드는 비용이 적게 들기 때문에 이런 상황은 실제로 자주 있음   
    
      
- 데이터를 label하는 것은 사람만이 가능하기에 sample을 labeling하는 데는 시간과 비용이 많이 소모되므로  
  label이 되지 않은 data보다 적을 일이 잦음

- 흥미로운 특성을 학습하기 위해 autoencoder를 강제하기 위해 coding layer의 크기를 제한하여 undercomplete를 만듦 
     
       
- 사실 다른 여러 종류의 제한을 사용 가능    
    
      
- coding layer의 크기를 입력 layer와 같게 하거나 혹은 더 크게 하여 overcomplete autoencoder를 만들 수 있음    

MNIST 분류하는 작은 신경망 만들기

In [ ]:
reset_graph()

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 150
n_outputs = 10

learning_rate = 0.01
l2_reg = 0.0005

activation = tf.nn.elu
regularizer = tf.contrib.layers.l2_regularizer(l2_reg)
initializer = tf.contrib.layers.variance_scaling_initializer()

X = tf.placeholder(tf.float32, shape=[None, n_inputs])
y = tf.placeholder(tf.int32, shape=[None])

weights1_init = initializer([n_inputs, n_hidden1])
weights2_init = initializer([n_hidden1, n_hidden2])
weights3_init = initializer([n_hidden2, n_outputs])

weights1 = tf.Variable(weights1_init, dtype=tf.float32, name="weights1")
weights2 = tf.Variable(weights2_init, dtype=tf.float32, name="weights2")
weights3 = tf.Variable(weights3_init, dtype=tf.float32, name="weights3")

biases1 = tf.Variable(tf.zeros(n_hidden1), name="biases1")
biases2 = tf.Variable(tf.zeros(n_hidden2), name="biases2")
biases3 = tf.Variable(tf.zeros(n_outputs), name="biases3")

hidden1 = activation(tf.matmul(X, weights1) + biases1)
hidden2 = activation(tf.matmul(hidden1, weights2) + biases2)
logits = tf.matmul(hidden2, weights3) + biases3

cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
reg_loss = regularizer(weights1) + regularizer(weights2) + regularizer(weights3)
loss = cross_entropy + reg_loss
optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(loss)

correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
pretrain_saver = tf.train.Saver([weights1, weights2, biases1, biases2])
saver = tf.train.Saver()

Regular training (without pretraining):

In [ ]:
n_epochs = 4
batch_size = 150
n_labeled_instances = 20000

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        n_batches = n_labeled_instances // batch_size
        for iteration in range(n_batches):
            print("\r{}%".format(100 * iteration // n_batches), end="")
            sys.stdout.flush()
            indices = rnd.permutation(n_labeled_instances)[:batch_size]
            X_batch, y_batch = mnist.train.images[indices], mnist.train.labels[indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        print("\r{}".format(epoch), "Train accuracy:", accuracy_val, end=" ")
        saver.save(sess, "./my_model_supervised.ckpt")
        accuracy_val = accuracy.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels})
        print("Test accuracy:", accuracy_val)

처음에 사용한 두 개의 autoencoder layer를 재사용 (reusing the first two layers of the autoencoder we pretrained)

In [ ]:
n_epochs = 4
batch_size = 150
n_labeled_instances = 20000

training_op = optimizer.minimize(loss, var_list=[weights3, biases3])  # Freeze layers 1 and 2 (optional)

with tf.Session() as sess:
    init.run()
    pretrain_saver.restore(sess, "./my_model_cache_frozen.ckpt")
    for epoch in range(n_epochs):
        n_batches = n_labeled_instances // batch_size
        for iteration in range(n_batches):
            print("\r{}%".format(100 * iteration // n_batches), end="")
            sys.stdout.flush()
            indices = rnd.permutation(n_labeled_instances)[:batch_size]
            X_batch, y_batch = mnist.train.images[indices], mnist.train.labels[indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        print("\r{}".format(epoch), "Train accuracy:", accuracy_val, end="\t")
        saver.save(sess, "./my_model_supervised_pretrained.ckpt")
        accuracy_val = accuracy.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels})
        print("Test accuracy:", accuracy_val)

## 👊 Stacked denoising Autoencoder

Note: the book uses `tf.contrib.layers.dropout()` rather than `tf.layers.dropout()` (which did not exist when this chapter was written). It is now preferable to use `tf.layers.dropout()`, because anything in the contrib module may change or be deleted without notice. The `tf.layers.dropout()` function is almost identical to the `tf.contrib.layers.dropout()` function, except for a few minor differences. Most importantly:
* you must specify the dropout rate (`rate`) rather than the keep probability (`keep_prob`), where `rate` is simply equal to `1 - keep_prob`,
* the `is_training` parameter is renamed to `training`.

- Autoencoder가 유용한 특성을 학습하도록 강제하는 다른 방법은 
  1. 입력에 잡음을 추가
  2. noise가 없는 원본 입력을 복원하도록 훈련 시키는 것   
     
         
           
- autoencoder가 입력을 출력으로 복사하지 못하므로 data에 있는 pattern을 찾아야 함

  Stacked Denoising Autoencoder (적층 잡음제거 오토인코더)

  noise는 입력에 추가된 순수한 Gaussian(가우시안) noise이거나 Drop out처럼 무작위로 입력을 꺼서 발생 시킬 수도 있음   

#### - Gaussian noise(left) & Dropout(right)을 사용한 Denoising Autoencoder

In [ ]:
from IPython.display import Image
Image(filename = "Ch 15. Gaussian noise(left) & Dropout(right)을 사용한 Denoising Autoencoder.jpg", width = 600, height = 600)

#### 1. Using Gaussian noise

In [ ]:
reset_graph()

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 150  # codings
n_hidden3 = n_hidden1
n_outputs = n_inputs

learning_rate = 0.01

In [ ]:
noise_level = 1.0

X = tf.placeholder(tf.float32, shape=[None, n_inputs])
X_noisy = X + noise_level * tf.random_normal(tf.shape(X))

hidden1 = tf.layers.dense(X_noisy, n_hidden1, activation=tf.nn.relu,
                          name="hidden1")
hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, # not shown in the book
                          name="hidden2")                            # not shown
hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=tf.nn.relu, # not shown
                          name="hidden3")                            # not shown
outputs = tf.layers.dense(hidden3, n_outputs, name="outputs")        # not shown

reconstruction_loss = tf.reduce_mean(tf.square(outputs - X)) # MSE

In [ ]:
optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(reconstruction_loss)
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [ ]:
n_epochs = 10
batch_size = 150

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        n_batches = mnist.train.num_examples // batch_size
        for iteration in range(n_batches):
            print("\r{}%".format(100 * iteration // n_batches), end="")
            sys.stdout.flush()
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch})
        loss_train = reconstruction_loss.eval(feed_dict={X: X_batch})
        print("\r{}".format(epoch), "Train MSE:", loss_train)
        saver.save(sess, "./my_model_stacked_denoising_gaussian.ckpt")

#### 2. Using dropout

In [ ]:
reset_graph()

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 150  # codings
n_hidden3 = n_hidden1
n_outputs = n_inputs

learning_rate = 0.01

In [ ]:
dropout_rate = 0.3

training = tf.placeholder_with_default(False, shape=(), name='training')

X = tf.placeholder(tf.float32, shape=[None, n_inputs])
X_drop = tf.layers.dropout(X, dropout_rate, training=training)

hidden1 = tf.layers.dense(X_drop, n_hidden1, activation=tf.nn.relu,
                          name="hidden1")
hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, # not shown in the book
                          name="hidden2")                            # not shown
hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=tf.nn.relu, # not shown
                          name="hidden3")                            # not shown
outputs = tf.layers.dense(hidden3, n_outputs, name="outputs")        # not shown

reconstruction_loss = tf.reduce_mean(tf.square(outputs - X)) # MSE

In [ ]:
optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(reconstruction_loss)
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [ ]:
n_epochs = 10
batch_size = 150

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        n_batches = mnist.train.num_examples // batch_size
        for iteration in range(n_batches):
            print("\r{}%".format(100 * iteration // n_batches), end="")
            sys.stdout.flush()
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, training: True})
        loss_train = reconstruction_loss.eval(feed_dict={X: X_batch})
        print("\r{}".format(epoch), "Train MSE:", loss_train)
        saver.save(sess, "./my_model_stacked_denoising_dropout.ckpt")

In [ ]:
show_reconstructed_digits(X, outputs, "./my_model_stacked_denoising_dropout.ckpt")

## 👊 Sparse Autoencoder (희소 Autoencoder)

### &nbsp;&nbsp;𝐎 정의

- 좋은 특성을 추출하도록 만드는 다른 제약의 방식인 Sparsity(희소)   
      
           

### &nbsp;&nbsp;𝐎 원리   
   
- cost function에 적절한 항을 추가하여 autoencoder가 coding layer에서 활성화되는 뉴런 수를 감소시키도록 만듦    
    
  => coding layer에서 평균적으로 5% neurons만 두드러지게 활성화되도록 강제 가능    
       
  => autoencoder가 적은 수의 활성화된 뉴런을 조합하여 입력을 표현해야 함     
                 
  coding layer의 각 neuron은 유용한 특성을 표현하게 됨.(수입이 매우 적으면 정말 필요한 곳에만 돈을 사용하게되는 것처럼)
  
  
- Sparsity Model을 만들기 위해 먼저 각 훈련 반복에서 coding layer의 실제 saprsity 정도를 측정해야 함  
  
      
- 이를 위해 전체 훈련 배치에 대해 coding layer의 평균적인 활성화를 계산  
      
         
- 배치 크기는 너무 작지 않아야하고 그렇지 않으면 평균값이 정확하지 않음
     
     
- 각 neurons에 대한 평균 활성화 정도를 알면 cost function에 sparsity loss를 추가하여 neurons이 너무 활성화되지 않도록 규제할 수 있음.  
  ex. 한 neurons의 평균 활성화가 0.3이고 목표 sparsity 정도가 0.1이라면 이 neurons은 덜 활성화되도록 규제되어야 함.   
     
      
- 간단한 방법은 
  1. 비용 함수에 제곱 오차 $(0.3 - 0.1)^{2}$을 추가하는 것
실전에서 더 좋은 방법은
  2. 아래 그래프에서 볼 수 있듯이 Kullback-Leibler Divergence(쿨백 라이블러 발산)를 사용하는 것.

In [ ]:
p = 0.1
q = np.linspace(0.001, 0.999, 500)
kl_div = p * np.log(p / q) + (1 - p) * np.log((1 - p) / (1 - q))
mse = (p - q)**2
plt.plot([p, p], [0, 0.3], "k:")
plt.text(0.05, 0.32, "Target\nsparsity", fontsize=14)
plt.plot(q, kl_div, "b-", label="KL divergence")
plt.plot(q, mse, "r--", label="MSE")
plt.legend(loc="upper left")
plt.xlabel("Actual sparsity")
plt.ylabel("Cost", rotation=0)
plt.axis([0, 1, 0, 0.95])
save_fig("sparsity_loss_plot")

### - Kullback-Leibler Divergence (KL 발산) 

$ D_{KL}(P||Q) = \sum_i{P(i)log\frac{P(i)}{Q(i)}}$

coding에서 neurons이 활성화될 목표 확률 p와 실제 확률 q(즉, 훈련 배치에 대한 평균 활성화) 사이의 발산을 측정  
따라서 KL Divergence는 

$ D_{KL} (p||q) = plog\frac{p}{q} + (1-p)log\frac{1-p}{1-q} $

coding layer의 각 neurons에 대해 sparsity loss을 계산했다면, 이 loss들을 모두 합해서 cost function의 결과에 더함   
   
sparsity loss와 reconstruction loss의 상대적 중요도를 제어하기 위해 sparsity loss에 sparsity weight hyperparameter를 곱함   
   
sparsity weight hyperparameter가 너무 크면 model이 목표 saprsity에 가깝게 되고  
sparsity weight hyperparameter가 너무 작으면 model이 sparsity를 거의 무시할 것이므로 어떤 흥미로운 특성도 학습하지 못할 것  

In [ ]:
reset_graph()

n_inputs = 28 * 28
n_hidden1 = 1000  # sparse codings
n_outputs = n_inputs

In [ ]:
def kl_divergence(p, q):
    # Kullback Leibler divergence
    return p * tf.log(p / q) + (1 - p) * tf.log((1 - p) / (1 - q))

learning_rate = 0.01
sparsity_target = 0.1
sparsity_weight = 0.2

X = tf.placeholder(tf.float32, shape=[None, n_inputs])            # not shown in the book

hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.sigmoid) # not shown
outputs = tf.layers.dense(hidden1, n_outputs)                     # not shown

hidden1_mean = tf.reduce_mean(hidden1, axis=0) # batch mean
sparsity_loss = tf.reduce_sum(kl_divergence(sparsity_target, hidden1_mean))
reconstruction_loss = tf.reduce_mean(tf.square(outputs - X)) # MSE
loss = reconstruction_loss + sparsity_weight * sparsity_loss

optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(loss)

In [ ]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [ ]:
n_epochs = 100
batch_size = 1000

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        n_batches = mnist.train.num_examples // batch_size
        for iteration in range(n_batches):
            print("\r{}%".format(100 * iteration // n_batches), end="")
            sys.stdout.flush()
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch})
        reconstruction_loss_val, sparsity_loss_val, loss_val = sess.run([reconstruction_loss, sparsity_loss, loss], feed_dict={X: X_batch})
        print("\r{}".format(epoch), "Train MSE:", reconstruction_loss_val, "\tSparsity loss:", sparsity_loss_val, "\tTotal loss:", loss_val)
        saver.save(sess, "./my_model_sparse.ckpt")

In [ ]:
show_reconstructed_digits(X, outputs, "./my_model_sparse.ckpt")

Note that the coding layer must output values from 0 to 1, which is why we use the sigmoid activation function:

In [ ]:
hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.sigmoid)

To speed up training, you can normalize the inputs between 0 and 1, and use the cross entropy instead of the MSE for the cost function:

In [ ]:
logits = tf.layers.dense(hidden1, n_outputs)
outputs = tf.nn.sigmoid(logits)

xentropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=X, logits=logits)
reconstruction_loss = tf.reduce_mean(xentropy)

## 👊 Variational Autoencoder (변이형 Autoencoder)

인기 많은 autoencoder의 한 종류   
   
기존 Autoencoder와 다른 점  
- Probabilistic Autoencoder. 즉, 훈련이 끝난 후에도 출력이 부분적으로 우연에 의해 결정됨  
&nbsp; (이와는 반대로 Denoising Autoencoder는 훈련 시에만 무작위성을 사용)   
     
     
- Generative Autoencoder라는 것이 중요.  마치 훈련 set에서 samping된 것 같은 new sample을 생성 가능  
  
=> 이러한 두 속성에 의해 RBM과 유사하게 되고 훈련이 더 쉽고 sampling 과정이 훨씬 빠름

#### Variational Autoencoder(left) & 이를 통과하는 Sample(right)

In [ ]:
from IPython.display import Image
Image(filename = "Ch 15. Variational Autoencoder(left) & 이를 통과하는 sample(right).jpg", width = 600, height = 600)

위 그림 왼쪽은 Variational Autoencoder를 보여줌   
1. 구조는 Encoder와 그 뒤를 따르는 Decoder로 구성  
2. 차이점은 입력에 대한 coding을 바로 만드는 대신 encoder가 mean coding(평균 코딩) $ \mu$와 표준편자 $\sigma$를 만듦  
3. 실제 coding은 평균이 $\mu$이고 표준편차가 $\sigma$인 Gaussian distribution에서 random하게 sampling된 후 decoder가 sampling된 coding을 보통처럼 decoding
4. 그림 15-11의 오른쪽은 autoencoder를 통과하는 훈련 샘플을 보여줌 먼저 encoder가 $\mu$와 $\sigma$를 만들면 coding이 random하게 선택됨  
($\mu$의 위치와 정확히 같지는 않음)  
5. 이 coding이 decode되고 최종 출력은 훈련 sample을 닮는다.

- 입력이 매우 복잡한 분포를 가지더라도 variational autoencoder는 마치 간단한 gaussian distribution에서 sampling된 것처럼 보이는 coding을 만드는 경향있음     
    
    
- 훈련하는 동안 cost function이 coding을 gaussian sample들의 군집처럼 보이는 거의 구 형태를 가진    
  coding space(코딩 공간) (또는 latent space 잠재 변수 공간)으로 점진적으로 이동시키므로     
  variational autoencoder는 훈련이 끝난 뒤 new sample을 매우 쉽게 생성할 수 있게 됨    
   
    
- Gaussian Distribution로부터 random한 coding을 sampling해 decoding하면 됨

- cost function은 두 부분으로 구성   
   
  1. Autoencoder가 입력을 재생산하도록 만드는 일반적인 reconstruction loss(재구성 손실), 앞에선 여기에 cross entropy를 사용
  2. Gaussian Distribution에서 sample된 것 같은 coding을 가지도록 autoencoder를 강제하는 latent loss(잠재 변수 손실)  
  3. 목표 분포(Gaussian Distribyution)와 실제 coding distribution 사이의 KL Divergence을 사용  
  4. coding layer으로 전달될 수 있는 정보 양을 제한함으로써 autoencoder가 유용한 특성을 학습하게 만드는 gaussian noise 때문에 수식이 복잡.

In [ ]:
reset_graph()

from functools import partial

n_inputs = 28 * 28
n_hidden1 = 500
n_hidden2 = 500
n_hidden3 = 20  # codings
n_hidden4 = n_hidden2
n_hidden5 = n_hidden1
n_outputs = n_inputs
learning_rate = 0.001

initializer = tf.contrib.layers.variance_scaling_initializer()

my_dense_layer = partial(
    tf.layers.dense,
    activation=tf.nn.elu,
    kernel_initializer=initializer)

X = tf.placeholder(tf.float32, [None, n_inputs])
hidden1 = my_dense_layer(X, n_hidden1)
hidden2 = my_dense_layer(hidden1, n_hidden2)
hidden3_mean = my_dense_layer(hidden2, n_hidden3, activation=None)
hidden3_sigma = my_dense_layer(hidden2, n_hidden3, activation=None)
noise = tf.random_normal(tf.shape(hidden3_sigma), dtype=tf.float32)
hidden3 = hidden3_mean + hidden3_sigma * noise
hidden4 = my_dense_layer(hidden3, n_hidden4)
hidden5 = my_dense_layer(hidden4, n_hidden5)
logits = my_dense_layer(hidden5, n_outputs, activation=None)
outputs = tf.sigmoid(logits)

xentropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=X, logits=logits)
reconstruction_loss = tf.reduce_sum(xentropy)

In [ ]:
eps = 1e-10 # smoothing term to avoid computing log(0) which is NaN
latent_loss = 0.5 * tf.reduce_sum(
    tf.square(hidden3_sigma) + tf.square(hidden3_mean)
    - 1 - tf.log(eps + tf.square(hidden3_sigma)))

In [ ]:
loss = reconstruction_loss + latent_loss

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [ ]:
n_epochs = 50
batch_size = 150

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        n_batches = mnist.train.num_examples // batch_size
        for iteration in range(n_batches):
            print("\r{}%".format(100 * iteration // n_batches), end="")
            sys.stdout.flush()
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch})
        loss_val, reconstruction_loss_val, latent_loss_val = sess.run([loss, reconstruction_loss, latent_loss], feed_dict={X: X_batch})
        print("\r{}".format(epoch), "Train total loss:", loss_val, "\tReconstruction loss:", reconstruction_loss_val, "\tLatent loss:", latent_loss_val)
        saver.save(sess, "./my_model_variational.ckpt")

In [ ]:
reset_graph()

from functools import partial

n_inputs = 28 * 28
n_hidden1 = 500
n_hidden2 = 500
n_hidden3 = 20  # codings
n_hidden4 = n_hidden2
n_hidden5 = n_hidden1
n_outputs = n_inputs
learning_rate = 0.001

initializer = tf.contrib.layers.variance_scaling_initializer()
my_dense_layer = partial(
    tf.layers.dense,
    activation=tf.nn.elu,
    kernel_initializer=initializer)

X = tf.placeholder(tf.float32, [None, n_inputs])
hidden1 = my_dense_layer(X, n_hidden1)
hidden2 = my_dense_layer(hidden1, n_hidden2)
hidden3_mean = my_dense_layer(hidden2, n_hidden3, activation=None)
hidden3_gamma = my_dense_layer(hidden2, n_hidden3, activation=None)
noise = tf.random_normal(tf.shape(hidden3_gamma), dtype=tf.float32)
hidden3 = hidden3_mean + tf.exp(0.5 * hidden3_gamma) * noise
hidden4 = my_dense_layer(hidden3, n_hidden4)
hidden5 = my_dense_layer(hidden4, n_hidden5)
logits = my_dense_layer(hidden5, n_outputs, activation=None)
outputs = tf.sigmoid(logits)

xentropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=X, logits=logits)
reconstruction_loss = tf.reduce_sum(xentropy)
latent_loss = 0.5 * tf.reduce_sum(
    tf.exp(hidden3_gamma) + tf.square(hidden3_mean) - 1 - hidden3_gamma)
loss = reconstruction_loss + latent_loss

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

### Generate digits

Let's train the model and generate a few random digits:

In [ ]:
import numpy as np

n_digits = 60
n_epochs = 50
batch_size = 150

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        n_batches = mnist.train.num_examples // batch_size
        for iteration in range(n_batches):
            print("\r{}%".format(100 * iteration // n_batches), end="") # not shown in the book
            sys.stdout.flush()                                          # not shown
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch})
        loss_val, reconstruction_loss_val, latent_loss_val = sess.run([loss, reconstruction_loss, latent_loss], feed_dict={X: X_batch}) # not shown
        print("\r{}".format(epoch), "Train total loss:", loss_val, "\tReconstruction loss:", reconstruction_loss_val, "\tLatent loss:", latent_loss_val)  # not shown
        saver.save(sess, "./my_model_variational.ckpt")  # not shown
    
    codings_rnd = np.random.normal(size=[n_digits, n_hidden3])
    outputs_val = outputs.eval(feed_dict={hidden3: codings_rnd})

In [ ]:
plt.figure(figsize=(8,50)) # not shown in the book
for iteration in range(n_digits):
    plt.subplot(n_digits, 10, iteration + 1)
    plot_image(outputs_val[iteration])

In [ ]:
n_rows = 6
n_cols = 10
plot_multiple_images(outputs_val.reshape(-1, 28, 28), n_rows, n_cols)
save_fig("generated_digits_plot")
plt.show()

Note that the latent loss is computed differently in this second variant:

In [ ]:
latent_loss = 0.5 * tf.reduce_sum(
    tf.exp(hidden3_gamma) + tf.square(hidden3_mean) - 1 - hidden3_gamma)

### Encode & Decode

Encode:

In [ ]:
n_digits = 3
X_test, y_test = mnist.test.next_batch(batch_size)
codings = hidden3

with tf.Session() as sess:
    saver.restore(sess, "./my_model_variational.ckpt")
    codings_val = codings.eval(feed_dict={X: X_test})

Decode:

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_variational.ckpt")
    outputs_val = outputs.eval(feed_dict={codings: codings_val})

Let's plot the reconstructions:

In [ ]:
fig = plt.figure(figsize=(8, 2.5 * n_digits))
for iteration in range(n_digits):
    plt.subplot(n_digits, 2, 1 + 2 * iteration)
    plot_image(X_test[iteration])
    plt.subplot(n_digits, 2, 2 + 2 * iteration)
    plot_image(outputs_val[iteration])

### Interpolate digits

In [ ]:
n_iterations = 3
n_digits = 6
codings_rnd = np.random.normal(size=[n_digits, n_hidden3])

with tf.Session() as sess:
    saver.restore(sess, "./my_model_variational.ckpt")
    target_codings = np.roll(codings_rnd, -1, axis=0)
    for iteration in range(n_iterations + 1):
        codings_interpolate = codings_rnd + (target_codings - codings_rnd) * iteration / n_iterations
        outputs_val = outputs.eval(feed_dict={codings: codings_interpolate})
        plt.figure(figsize=(11, 1.5*n_iterations))
        for digit_index in range(n_digits):
            plt.subplot(1, n_digits, digit_index + 1)
            plot_image(outputs_val[digit_index])
        plt.show()

# 8. Autoencoder 종류

- 수축 오토인코더 (Contractive Autoencoder, CAE)
  - 훈련하는 동안 입력에 대한 coding의 변화율이 작도록 제약을 받음    
 (두 개의 비슷한 입력은 비슷한 coding이 되어야함)   
   
   <p>   
- 적층 합성곱 오토인코더 (Stacked Convoluional Autoencoders)
  - Convolution layer을 통해 처리되는 image를 재구성함으로써 시각적 특징을 추출하는 법을 학습   
     
   <p>
- 확률적 생성 네트워크 (Generative Stochastic Network, GSN)
  - Data를 생성하는 기능을 추가한 denoise autoencoder의 일반화된 model  
  
  <p>
- WTA 오토인코더 (Winner-Take-All Autoencoder)
  - 훈련하는 동안 coding layer에 있는 모든 neurons의 활성화를 계산한 후 훈련 배치에서 각 neurons에 대해 최대 $k%$활성화만 보존하고 나머지는 0으로 설정.  
  - 자연스럽게 sparsity coding을 만들고 비슷한 방식으로 WTA 방식을 사용하여 sparsity convolution autoencoder를 생성  
     
   <p> 
- 적대적 생성 네트워크 (Generative Adversarial Network, GAN)
  - 'discriminator'(판별자)라 불리는 network가 'generator'(생성자)라고 불리는 두 번째 newwork가 만든 가짜 data와 실제 data를 구분하도록 훈련됨
  - generator는 discriminator를 속이는 법을 학습하며, discriminator는 generator의 속임수를 피하는 법을 학습
  - 이런 경쟁은 매우 현실적인 가짜 data와 안정적인 coding을 생성하도록 만듦
  - 적대적 훈련은 많은 관심을 받고 있음